In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
from IPython.display import display
import scipy as sp

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

pd.options.display.max_columns = None   # To see the entire columns of dataframes
pd.options.display.max_rows = 100    


In [2]:
train_combat = pd.read_csv("data/train_combat.csv")
train_payment = pd.read_csv("data/train_payment.csv")
train_pledge = pd.read_csv("data/train_pledge.csv")
train_trade = pd.read_csv("data/train_trade.csv")
train_activity = pd.read_csv("data/train_activity.csv")
train_label = pd.read_csv("data/train_label.csv")

In [3]:
# Copy files in case of retrieving the original ones
combat = train_combat.copy()
pledge = train_pledge.copy()
payment = train_payment.copy()
trade = train_trade.copy()
activity = train_activity.copy()

In [4]:
# Total amount spent per account the whole period
train_label["total_spent"] = train_label["survival_time"] * train_label["amount_spent"]
label = train_label["acc_id"]

In [5]:
# No need to keep the rows whose "acc_id" does not appear in Label data
combat = combat[combat["acc_id"].isin(label)]
pledge = pledge[pledge["acc_id"].isin(label)]
payment = payment[payment["acc_id"].isin(label)]
trade = trade[trade["source_acc_id"].isin(label)]
trade = trade[trade["target_acc_id"].isin(label)]
activity = activity[activity["acc_id"].isin(label)]

## Common Features

In [6]:
# The number of characters per account
all_char = pd.concat([train_combat[["char_id", "acc_id"]], train_pledge[["char_id", "acc_id"]], train_activity[["char_id", "acc_id"]]])
char_count = all_char.groupby("char_id")["acc_id"].count()
char_count = pd.DataFrame({"char_count" : char_count})  # in case of merging

In [7]:
# Find all the unique servers
total_server = pd.concat([train_combat["server"], train_pledge["server"], train_trade["server"], train_activity["server"]])
server_list = total_server.unique()

# Encode servers into integer values
le = LabelEncoder()
le.fit(server_list)
for df in [combat, pledge, trade, activity]:
    df["server"] = le.transform(df["server"])

## Trade

In [8]:
trade.rename(columns={"source_acc_id" : "acc_id"}, inplace=True)

# Remove "source_char_idd" and "target_char_id"
trade.drop(["source_char_id", "target_char_id"], axis=1, inplace=True)

# Substitute NaNs with median at "item_price"
trade["item_price"].fillna(trade["item_price"].median(), inplace=True)

# Categorize transaction time
bins = [0, 60000, 120000, 180000, 239999]
bin_label = [0, 1, 2, 3]   
trade["time_bin"] = pd.cut(pd.to_numeric(trade["time"].str.replace(":", "")), bins=bins, labels=bin_label)
# Those who play during the time other than evening must be differentiated
trade["time_bin"] = trade["time_bin"].map({0:1, 1:1, 2:1, 3:0}) 
trade.drop("time", axis=1, inplace=True)

# Total price per each trade
trade["total_item_price"] = trade["item_amount"] * trade["item_price"]

# Make two trade DataFrames for each source and target
source_trade = trade.drop("target_acc_id", axis=1)
target_trade = trade.drop("acc_id", axis=1)
target_trade.rename(columns={"target_acc_id" : "acc_id"}, inplace=True)

# Total number of trade occurence per account the whole period
source_trade_count = source_trade["acc_id"].value_counts()
target_trade_count = target_trade["acc_id"].value_counts()

source_trade_count = pd.DataFrame({"acc_id" : source_trade_count.index,
                                   "count" : source_trade_count})    # Make a dataframe to merge based on "acc_id"
target_trade_count = pd.DataFrame({"acc_id" : target_trade_count.index,
                                   "count" : target_trade_count})

trade_count = pd.merge(source_trade_count, target_trade_count, on = "acc_id")
trade_count["count"] = trade_count["count_x"] + trade_count["count_y"]
trade_count.drop(["count_x", "count_y"], axis=1, inplace=True)

# Remove columns that do not seem necessary
trade.drop(["server", "target_acc_id", "item_type", "item_amount", "item_price"], axis=1, inplace=True)

trade = pd.concat(
    [pd.pivot_table(data=trade, index=["acc_id", "day"], values=["time_bin", "total_item_price"], aggfunc=sum), # sum all the prices
    pd.pivot_table(data=trade, index=["acc_id", "day"], values=["type"])], # mean for trade type
    axis=1)

## Pledge

In [9]:
pledge.columns

Index(['day', 'acc_id', 'char_id', 'server', 'pledge_id', 'play_char_cnt',
       'combat_char_cnt', 'pledge_combat_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'same_pledge_cnt', 'temp_cnt', 'etc_cnt',
       'combat_play_time', 'non_combat_play_time'],
      dtype='object')

### 'play_char_cnt'
혈맹 내 유저의 접속률이 높을수록 생존에 영향이 있을거라 예상  
--> 접속률에 따른 점수 부여. 접속률 1위 혈맹에 1.0 , 나머지 혈맹이 1위 혈맹 대비 비례한 점수 부여.

In [10]:
# 각 혈맹의 평균 유저 접속률 지표
pledge_pivot = pd.pivot_table(data=pledge, index=['pledge_id'], values='play_char_cnt', aggfunc='mean')

# 정렬 후 1위 값으로 나눔 => 접속률 1위인 혈맹 1.0, 나머지 1.0보다 차례로 낮은 값.
avg_play_rate_per_pledge = pledge_pivot.play_char_cnt.sort_values(ascending=False)\
                                    / pledge_pivot.play_char_cnt.sort_values(ascending=False).iloc[0]

In [11]:
to_be_merged = pd.DataFrame({'avg_play_rate_per_pledge' : avg_play_rate_per_pledge})
to_be_merged.head(10)

,avg_play_rate_per_pledge
pledge_id,
17429,1.000000
7891,0.958941
27662,0.941473
3703,0.939236
33924,0.934305
23128,0.873290
4499,0.836410
9652,0.825830
28873,0.784145


In [12]:
pledge = pd.merge(pledge, to_be_merged, on='pledge_id')

In [13]:
# 접속률 1위 혈맹 확인
pledge[pledge.pledge_id==17429]

,day,acc_id,char_id,server,pledge_id,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt,random_defender_cnt,same_pledge_cnt,temp_cnt,etc_cnt,combat_play_time,non_combat_play_time,avg_play_rate_per_pledge
143906,1,34280,241743,4,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143907,1,58149,209756,4,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143908,1,120498,236119,4,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143909,1,58351,61415,4,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143910,1,78825,26046,4,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143911,1,12896,160650,4,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143912,1,23955,99071,4,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143913,1,119848,136484,4,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143914,1,27558,392408,4,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143915,1,27558,45153,4,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0


### pledge_combat_cnt
마찬가지로 혈맹간 전투(공성 등) 같은 엔드 컨텐츠를 많이 즐기는 혈맹의 유저일 수록 생존률이 높을거라 예상  
--> 혈맹간 전투 횟수에 점수 부여. 위의 접속률과 같은 방식으로 점수 부여.

In [14]:
# 혈맹간 총 전투 수 체크
pledge_pivot = pd.pivot_table(data=pledge, index=['pledge_id'], values='pledge_combat_cnt', aggfunc='sum')

total_combat_cnt_per_pledge = pledge_pivot.pledge_combat_cnt.sort_values(ascending=False)\
                                / pledge_pivot.pledge_combat_cnt.sort_values(ascending=False).iloc[0]

In [15]:
to_be_merged = pd.DataFrame({'total_combat_cnt_per_pledge':total_combat_cnt_per_pledge})
to_be_merged.head(10)

,total_combat_cnt_per_pledge
pledge_id,
36551,1.000000
13214,0.560333
38266,0.459779
36909,0.447103
9427,0.416330
21739,0.345535
17546,0.314619
17834,0.308721
10521,0.264475


In [16]:
pledge = pd.merge(pledge, to_be_merged, on='pledge_id')

In [17]:
# 혈맹간 전투 수 1위 혈맹 확인
pledge[pledge.pledge_id==36551]

,day,acc_id,char_id,server,pledge_id,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt,random_defender_cnt,same_pledge_cnt,temp_cnt,etc_cnt,combat_play_time,non_combat_play_time,avg_play_rate_per_pledge,total_combat_cnt_per_pledge
428286,1,86413,35092,14,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428287,1,128409,20809,14,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428288,1,57985,1233,14,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428289,1,117569,202916,14,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428290,1,42561,395794,14,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428291,1,82662,377980,14,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428292,1,40748,80418,14,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428293,1,12203,298219,14,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428294,1,44591,241221,14,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428295,1,89162,287358,14,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0


In [18]:
# The number of members in each pledge
pledge_num_people = pledge["pledge_id"].value_counts()

# Remove "char_id", "server", "pledge_id" (doesn't seem to matter)
pledge.drop(["char_id", "server", "pledge_id"], axis=1, inplace=True)

# Sum all the values according to days per account
pledge = pd.concat([pd.pivot_table(data=pledge.drop(['avg_play_rate_per_pledge','total_combat_cnt_per_pledge'], axis=1), index=["acc_id", "day"], aggfunc=sum),
                   pd.pivot_table(data=pledge, index=['acc_id','day'], values=['avg_play_rate_per_pledge','total_combat_cnt_per_pledge'])], axis=1)

# Change name of the columns which are same with the ones in combat file
rename_dict = {"etc_cnt" : "p_etc_cnt", "random_attacker_cnt" : "p_random_attacker_cnt",
              'same_pledge_cnt' : 'p_same_pledge_cnt', "temp_cnt" : "p_temp_cnt", 
               "random_defender_cnt" : "p_random_defender_cnt"}
pledge.rename(columns=rename_dict, inplace=True)

## Activity

In [19]:
# Remove "char_id"
activity.drop("char_id", axis=1, inplace=True)

# Total play time per account the whole period
total_play = activity.groupby("acc_id")["playtime"].sum()
total_play = pd.DataFrame({"total_play_time" : total_play})  # in case of merging

# activity["cum_play_time"] = activity.groupby("acc_id")["playtime"].cumsum()

# Sum all the values per day for each account
activity = pd.concat([pd.pivot_table(data=activity.drop("server", axis=1), index=["acc_id", "day"], aggfunc=sum),
                     pd.pivot_table(data=activity, index=["acc_id", "day"], values=["server"])],
                    axis=1)

# Total exp
activity["total_exp"] = activity["solo_exp"] + activity["party_exp"] + activity["quest_exp"]

# How long do users spend time fishing?
activity["fishing_prop"] = activity["fishing"] / activity["playtime"]

In [20]:
activity.columns

Index(['death', 'enchant_count', 'exp_recovery', 'fishing',
       'game_money_change', 'npc_kill', 'party_exp', 'playtime',
       'private_shop', 'quest_exp', 'revive', 'rich_monster', 'solo_exp',
       'server', 'total_exp', 'fishing_prop'],
      dtype='object')

## Combat

In [21]:
combat.columns

Index(['day', 'acc_id', 'char_id', 'server', 'class', 'level', 'pledge_cnt',
       'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent'],
      dtype='object')

### class
"많은 사람들이 플레이 하는 주류 클래스 --> 성능이 좋은 클래스"  
"성능이 좋은 클래스를 하는 유저 --> 생존률 높지 않을까?"

|범주|직업|
|:---:|:---:|
|0|군주|
|1|기사|
|2|요정|
|3|마법사|
|4|다크엘프|
|5|용기사|
|6|환술사|
|7|전사|

In [22]:
combat['class'] = combat['class'].astype('category')

In [23]:
# 클래스별 전체 비율 조사
prop_class = combat['class'].value_counts() / combat['class'].value_counts().sum()
prop_class

1    0.208301
2    0.170270
3    0.156541
7    0.137134
4    0.135188
5    0.084303
0    0.081987
6    0.026275
Name: class, dtype: float64

In [24]:
# 전체 누적합 60% 미만을 차지하는 주류 클래스 조사
class60 = prop_class[~(prop_class.cumsum()>0.6)].index # 기사, 요정, 마법사 (70%로 하면 전사까지.)

def isMajorClass(classs):
    if classs in class60:
        return 1
    else:
        return 0

combat['isMajorClass'] = combat['class'].apply(isMajorClass)
combat['isMajorClass'].head(10)

0    1
1    1
2    1
3    0
4    1
5    1
6    0
7    1
8    1
9    0
Name: isMajorClass, dtype: int64

In [25]:
# Remove "char_id", "class", "server"
combat.drop(["char_id", "class", "server"], axis=1, inplace=True)

# Max level? or Mean level?
max_level = combat.groupby("acc_id")["level"].max()
mean_level = combat.groupby("acc_id")["level"].mean()
combat.drop("level", axis=1, inplace=True)

# Sum all the values per day for each account
combat = pd.concat([pd.pivot_table(data=combat.drop('isMajorClass', axis=1), index=["acc_id", "day"], aggfunc=sum),
                   pd.pivot_table(data=combat, index=['acc_id', 'day'], values='isMajorClass', aggfunc='mean')], axis=1)
# isMajorClass mean 처리해서 주류 클래스만 플레이했을 경우 1.0, 그 외에는 비주류 클래스 캐릭터 수에 따라 평균값 떨어짐

In [26]:
combat

etc_cnt  num_opponent  pledge_cnt  random_attacker_cnt  \
acc_id day                                                             
2      1     0.000000      0.000000    0.000000                  0.0   
       2     0.000000      0.000000    0.000000                  0.0   
       3     0.000000      0.000000    0.000000                  0.0   
       4     0.000000      0.000000    0.000000                  0.0   
       5     0.000000      0.000000    0.000000                  0.0   
       6     0.000000      0.000000    0.000000                  0.0   
       7     0.000000      0.000000    0.000000                  0.0   
       8     0.000000      0.000000    0.000000                  0.0   
       9     0.000000      0.000000    0.000000                  0.0   
       10    0.000000      0.000000    0.000000                  0.0   
       11    0.000000      0.000000    0.000000                  0.0   
       12    0.000000      0.000000    0.000000                  0.0   
       13    0.000000      0.000000    0.000000                  0.0   
       14    0.000000      0.000000    0.000000                  0.0   
       15    0.000000      0.000000    0.000000                  0.0   
       16    0.000000      0.000000    0.000000                  0.0   
       17    0.000000      0.000000    0.000000                  0.0   
       18    0.000000      0.000000    0.000000                  0.0   
       19    0.000000      0.000000    0.000000                  0.0   
       20    0.000000      0.000000    0.000000                  0.0   
       21    0.000000      0.000000    0.000000                  0.0   
       22    0.000000      0.000000    0.000000                  0.0   
       23    0.000000      0.000000    0.000000                  0.0   
       24    0.000000      0.000000    0.000000                  0.0   
       25    0.000000      0.000000    0.000000                  0.0   
       26    0.000000      0.000000    0.000000                  0.0   
       27    0.000000      0.000000    0.000000                  0.0   
       28    0.000000      0.000000    0.000000                  0.0   
5      9     0.000000      0.000000    0.000000                  0.0   
       14    0.000000      0.000000    0.000000                  0.0   
       20    0.000000      0.000000    0.000000                  0.0   
       22    0.000000      0.098129    0.000000                  0.0   
       23    0.000000      0.000000    0.000000                  0.0   
       24    0.000000      0.000000    0.000000                  0.0   
       25    0.000000      0.000000    0.000000                  0.0   
       26    0.000000      0.000000    0.000000                  0.0   
       27    0.000000      0.000000    0.000000                  0.0   
       28    0.000000      0.000000    0.000000                  0.0   
8      1     0.000000      0.000000    0.000000                  0.0   
       2     0.000000      0.000000    0.000000                  0.0   
       3     0.000000      0.000000    0.000000                  0.0   
       4     0.000000      0.000000    0.000000                  0.0   
       5     0.000000      0.098129    0.000000                  0.0   
       6     0.000000      0.000000    0.000000                  0.0   
       7     0.000000      0.098129    0.000000                  0.0   
       8     0.000000      0.000000    0.000000                  0.0   
       9     0.000000      0.000000    0.000000                  0.0   
       10    0.000000      0.000000    0.000000                  0.0   
       11    0.000000      0.000000    0.000000                  0.0   
       12    0.225223      0.392515    0.000000                  0.0   
...               ...           ...         ...                  ...   
130470 5     1.351338      0.785030    0.000000                  0.0   
       6     0.000000      0.000000    0.000000                  0.0   
       7     1.801784      1.079416    0.770211       

## Payment

In [27]:
# total_payment = payment.groupby("acc_id")["amount_spent"].sum()
# total_payment.head()

In [28]:
train_label[train_label["acc_id"] == 8]

###### Why are they different????? ####

,acc_id,survival_time,amount_spent,total_spent
5494,8,64,0.02031,1.299843


In [29]:
# Check the results
display(combat.tail())
display(payment.head())
display(pledge.head())
display(trade.head())
display(activity.head())

etc_cnt  num_opponent  pledge_cnt  random_attacker_cnt  \
acc_id day                                                            
130473 24   0.000000      0.000000    0.000000                  0.0   
       25   0.000000      1.079416    1.026948                  0.0   
       26   0.225223      5.985854    7.317004                  0.0   
       27   0.000000      0.000000    0.000000                  0.0   
       28   0.000000      0.000000    0.000000                  0.0   

            random_defender_cnt  same_pledge_cnt  temp_cnt  isMajorClass  
acc_id day                                                                
130473 24                   0.0              0.0  0.000000           0.0  
       25                   0.0              0.0  0.479728           0.0  
       26                   0.0              0.0  0.000000           0.0  
       27                   0.0              0.0  0.000000           0.0  
       28                   0.0              0.0  0.000000           0.0

,day,acc_id,amount_spent
0,18,27835,0.826123
1,23,27835,0.011735
2,27,27835,0.035204
3,17,12351,0.528062
4,11,125437,0.633674


combat_char_cnt  combat_play_time  p_etc_cnt  \
acc_id day                                                 
5      9           0.252625          0.399849   0.120331   
       14          0.180447          0.441895   0.219426   
       20          0.108268          0.593982   0.056626   
       22          0.252625          0.610198   0.092017   
       23          0.036089          0.339385   0.000000   

            non_combat_play_time  play_char_cnt  pledge_combat_cnt  \
acc_id day                                                           
5      9                0.000000       0.649608                0.0   
       14               0.000000       0.396983                0.0   
       20               0.000000       0.505250                0.0   
       22               0.000695       0.721786                0.0   
       23               0.001274       0.541340                0.0   

            p_random_attacker_cnt  p_random_defender_cnt  p_same_pledge_cnt  \
acc_id day                                                                    
5      9                 0.000000               0.143063                0.0   
       14                0.000000               0.000000                0.0   
       20                0.000000               0.071531                0.0   
       22                0.548695               0.357657                0.0   
       23                0.710075               0.000000                0.0   

            p_temp_cnt  avg_play_rate_per_pledge  total_combat_cnt_per_pledge  
acc_id day                                                                     
5      9      0.692181                  0.078765                     0.000021  
       14     0.197766                  0.078765                     0.000021  
       20     0.296649                  0.078765                     0.000021  
       22     0.098883                  0.030899                     0.000007  
       23     0.000000                  0.030899                     0.000007

time_bin  total_item_price  type
acc_id day                                  
2      1         3.0      1.702263e-06   0.0
       2         1.0      5.833293e-03   1.0
       3         1.0      1.138403e-06   0.0
       5         0.0      1.821335e-09   0.0
       7         1.0      1.433993e-06   0.0

death  enchant_count  exp_recovery  fishing  game_money_change  \
acc_id day                                                                   
2      1      0.0            0.0           0.0      0.0           1.024810   
       2      0.0            0.0           0.0      0.0          -1.044201   
       3      0.0            0.0           0.0      0.0           0.451317   
       4      0.0            0.0           0.0      0.0          -0.007230   
       5      0.0            0.0           0.0      0.0           0.015398   

            npc_kill  party_exp  playtime  private_shop  quest_exp  revive  \
acc_id day                                                                   
2      1         0.0        0.0  2.703457      2.462447        0.0     0.0   
       2         0.0        0.0  3.363522      3.149118        0.0     0.0   
       3         0.0        0.0  3.370544      3.183447        0.0     0.0   
       4         0.0        0.0  3.370544      3.131992        0.0     0.0   
       5         0.0        0.0  3.370544      3.135698        0.0     0.0   

            rich_monster  solo_exp  server  total_exp  fishing_prop  
acc_id day                                                           
2      1               0       0.0    20.0        0.0           0.0  
       2               0       0.0    20.0        0.0           0.0  
       3               0       0.0    20.0        0.0           0.0  
       4               0       0.0    20.0        0.0           0.0  
       5               0       0.0    20.0        0.0           0.0

In [30]:
# Squeeze the whole dataframes into one
df = combat.join(pledge).join(trade).join(activity)

In [31]:
df.isnull().sum()

etc_cnt                             0
num_opponent                        0
pledge_cnt                          0
random_attacker_cnt                 0
random_defender_cnt                 0
same_pledge_cnt                     0
temp_cnt                            0
isMajorClass                        0
combat_char_cnt                196818
combat_play_time               196818
p_etc_cnt                      196818
non_combat_play_time           196818
play_char_cnt                  196818
pledge_combat_cnt              196818
p_random_attacker_cnt          196818
p_random_defender_cnt          196818
p_same_pledge_cnt              196818
p_temp_cnt                     196818
avg_play_rate_per_pledge       196818
total_combat_cnt_per_pledge    196818
time_bin                       763721
total_item_price               763721
type                           763721
death                               0
enchant_count                       0
exp_recovery                        0
fishing     

In [32]:
df[100:200]

etc_cnt  num_opponent  pledge_cnt  random_attacker_cnt  \
acc_id day                                                            
20     7    0.000000      0.098129    0.256737                  0.0   
       8    0.000000      0.000000    0.000000                  0.0   
       9    6.193634      2.453219    0.128368                  0.0   
       10   0.112612      0.098129    0.000000                  0.0   
       11   0.000000      0.000000    0.000000                  0.0   
       12   0.000000      0.098129    0.128368                  0.0   
       13   3.941403      2.649476    0.513474                  0.0   
       14   0.000000      0.000000    0.000000                  0.0   
       15   0.000000      0.000000    0.000000                  0.0   
       16   1.013504      0.981288    0.064184                  0.0   
       17   0.000000      0.000000    0.000000                  0.0   
       18   0.000000      0.000000    0.000000                  0.0   
       20   0.000000      0.000000    0.000000                  0.0   
       21   0.900892      0.785030    0.000000                  0.0   
       22   0.225223      0.294386    0.064184                  0.0   
       23   0.000000      0.000000    0.000000                  0.0   
       24   0.000000      0.000000    0.000000                  0.0   
       25   0.000000      0.000000    0.000000                  0.0   
       26   0.000000      0.000000    0.000000                  0.0   
       27   3.828792      2.256961    0.128368                  0.0   
       28   0.112612      0.294386    0.000000                  0.0   
21     2    0.000000      0.098129    0.000000                  0.0   
       3    0.000000      0.098129    0.000000                  0.0   
       4    0.112612      0.392515    0.000000                  0.0   
       5    0.000000      0.000000    0.000000                  0.0   
       6    0.000000      0.000000    0.000000                  0.0   
       7    0.112612      0.098129    0.000000                  0.0   
       8    0.000000      0.000000    0.000000                  0.0   
       9    0.000000      0.000000    0.000000                  0.0   
       10   0.000000      0.000000    0.000000                  0.0   
       11   0.000000      0.000000    0.000000                  0.0   
       12   0.000000      0.000000    0.000000                  0.0   
       13   0.000000      0.000000    0.000000                  0.0   
       14   0.000000      0.000000    0.000000                  0.0   
       15   0.000000      0.000000    0.000000                  0.0   
       16   0.000000      0.000000    0.000000                  0.0   
       17   0.000000      0.000000    0.000000                  0.0   
       18   0.000000      0.000000    0.000000                  0.0   
       19   0.000000      0.000000    0.000000                  0.0   
       20   0.000000      0.000000    0.000000                  0.0   
       21   0.000000      0.098129    0.000000                  0.0   
       22   0.000000      0.000000    0.000000                  0.0   
       23   0.000000      0.000000    0.000000                  0.0   
       24   0.000000      0.000000    0.000000                  0.0   
       25   0.000000      0.000000    0.000000                  0.0   
       26   0.000000      0.000000    0.000000                  0.0   
       28   0.000000      0.000000    0.000000                  0.0   
31     1    0.112612      0.098129    0.000000                  0.0   
       2    0.000000      0.000000    0.000000                  0.0   
       3    0.000000      0.098129    0.000000                  0.0   
       4    0.225223      0.196258    0.000000                  0.0   
       5    0.788281      0.588773    0.000000                  0.0   
       6    0.900892      0.392515    0.000000                  0.0   
       7    1.238727      0.981288    0.000000                  0.0   
       8    0.900892      0.686901   

In [39]:
# '196818' 개의 NaN 값 생성 이유:
# 해당 유저가 '혈맹 전투 관련 활동'을 안한 날짜 존재.
# 평균으로 채워줄지, 0으로 채워줄지 고민
display(df.loc[38].total_combat_cnt_per_pledge)
display(pledge.loc[38].total_combat_cnt_per_pledge)

day
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16         NaN
18         NaN
19         NaN
20         NaN
21    0.000292
22    0.000292
23    0.000292
24    0.000292
25    0.000292
26    0.000292
27    0.000292
28    0.000292
Name: total_combat_cnt_per_pledge, dtype: float64

day
21    0.000292
22    0.000292
23    0.000292
24    0.000292
25    0.000292
26    0.000292
27    0.000292
28    0.000292
Name: total_combat_cnt_per_pledge, dtype: float64